# Simulation Control Notebook

This notebook will handle all of the scripting that would typically be done in bash but I decided to try in python to keep in with the primary theme
This should also help a TON with the math and such since bash doesn't do decimal arithmetic by itself.

In [1]:
import os
import numpy as np
import time
from subprocess import call
from datetime import datetime

In [2]:
directory_length = len(os.listdir("NewRunOutputs"))
directory = "NewRunOutputs/output"+str(directory_length)
os.mkdir(directory)
os.chdir("/home/jupyter-rubin/flashDosimetrySlim")

In [6]:
start = time.time()
total_energy = 7
for energy in range(1, total_energy, 1):
    for height in range(3, 4, 1):   
        for angle in range(5, 6, 1):
            # Changes the dimensions and alignments of detector and beam
            call(f'sed -i "s:/gate/sourceBlock/placement/setTranslation.*:/gate/sourceBlock/placement/setTranslation         0 {height} 0 cm:" GDet_PPSc_v1_0.mac', shell=True)
            call(f'sed -i "s:/gate/source/BeamS/gps/radius.*:/gate/source/BeamS/gps/radius {angle/10} cm" GDet_PPSc_v1_0.mac', shell=True)
            call(f'sed -i "s:/gate/application/setTotalNumberOfPrimaries.*:/gate/application/setTotalNumberOfPrimaries {10**energy}:" GDet_PPSc_v1_0.mac', shell=True)
            # mades a zero padded string version of the dimensions
            pad_height = str(height).zfill(2)
            pad_energy = str(energy).zfill(2)

            # edits the root name of the output file to represent the correct width height and angle
            call(f'sed -i "s:/gate/actor/initEdep/save .*:/gate/actor/initEdep/save  {directory}/h{pad_height}_e{pad_energy}_a{angle/10}_edep.txt:" GDet_PPSc_v1_0.mac', shell=True)
            call(f'sed -i "s:/gate/actor/ParticleCounter2/save .*:/gate/actor/ParticleCounter2/save  {directory}/h{pad_height}_e{pad_energy}_a{angle/10}_particle_counter_2.txt:" GDet_PPSc_v1_0.mac', shell=True)
            call(f'sed -i "s:/gate/actor/ParticleCounter1/save .*:/gate/actor/ParticleCounter1/save  {directory}/h{pad_height}_e{pad_energy}_a{angle/10}_particle_counter_1.txt:" GDet_PPSc_v1_0.mac', shell=True)
            call('Gate GDet_PPSc_v1_0.mac', shell=True)
end = time.time()
print(end - start)

KeyboardInterrupt: 

## Summation

In [ ]:
folder_path = "/home/jupyter-rubin/flashDosimetrySlim/"+directory
os.mkdir(folder_path+"/summation")

In [ ]:
def particle_count_sum(file_path):
    with open(file_path, "r") as file_1:
        remove_header_1 = file_1.readlines()[6:]
        list_of_nums_1 = "".join(remove_header_1).split()
        data_1 = [int(i) for i in list_of_nums_1]
        sum_1 = sum(data_1)
        
    total = sum_1
    return total

In [ ]:
def edep_dose_sum(file_path):
    with open(file_path, "r") as file:
        remove_header = file.readlines()[6:]
        list_of_nums = "".join(remove_header).split()
        data = [float(i) for i in list_of_nums]
        total = sum(data)
    
    return total

In [ ]:
def total_sums(folder_path, energy_levels):
    total_particles = [0] * energy_levels
    total_edep = [0] * energy_levels
        
    for file in os.listdir(folder_path):
        if "particle_counter" in file:
            total_particles[int(file[5:7])-1] += particle_count_sum(folder_path+"/"+file)
        if "edep-Dose" in file:
            total_edep[int(file[5:7])-1] += edep_dose_sum(folder_path+"/"+file)
    
    os.chdir(folder_path + "/summation")
    file = open("summation.txt", "a")
    
    for i in range(energy_levels):
        print(total_particles[i],total_edep[i])
        file.write(str(total_particles[i]) + '\t' + str(total_edep[i]) + '\n')
        
    file.close()

In [ ]:
total_sums(folder_path, total_energy-1)

### Sed
`sed` is a tool developed for text manipulation of files. It is **extremely** fast and light weight. It is also a product of the early days of computing so following with sterotypes it is 'clunky and fragile' to use. I will walk you through what is going on.

- First:
`sed` stands for "stream editor" which makes sense since all it does is edit sequences of text in streams. `sed -i` tells `sed` to do its thing **Inline**. Normally it would "open" the file to memory, do its edits on the file in memory then print it out to standard out or the user would redirect the printing to another file. This is useful but not what we want, we want the file to be edited right then and there.

- Second
`s:` stands for substitution, `sed` matches patterns of text based on **regular expressions**. The pattern following `:` is the pattern being looked for, the `.*` matches any amount of any character its a super wild card. The stuff following the second `:` is what the first pattern will be replaced or substituted with.
- This is where Python makes this easy, I'm using an fstring to create a string and put the value of the variable width divided by 2 and with 2 decimals in the location it appears in the string. I get easy formatting and string manipulation unlike bash
The final `:` denotes the end of the `sed` comand and the `"` maks the end of the pattern stuff to the command. The patterns must be passed to `sed` as strings which is why I am using `"`.

- Third
The last part of the string `mac/run.mac` is the file being changed by `sed`